In [0]:
# %sql
# WITH autopay_latesttransaction AS (
# WITH cleaned_data AS (
#   SELECT 
#     TRIM(REPLACE(REPLACE(REPLACE(POLICY_NUMBER, ',' , ''), ' ', ''), '.', '')) AS POLICY_NUMBER,
#     REMARKS,
#     TRANSACTION_DATE,
#     STATUS,
#     SOURCE,
#     BILLING_TYPE,
#     CARD_BRAND,
#     CARD_TYPE,
#     CARD_LEVEL,
#     BANK
#   FROM curated_catalog.analytics.autopaybilling_daily
# ),
# label_latest_transaction AS (
#   SELECT 
#     POLICY_NUMBER AS POLICY_NUMBER,
#     REMARKS,
#     TRANSACTION_DATE,
#     STATUS,
#     SOURCE,
#     BILLING_TYPE,
#     CARD_BRAND,
#     CARD_TYPE,
#     CARD_LEVEL,
#     BANK,
#     ROW_NUMBER() OVER (PARTITION BY POLICY_NUMBER ORDER BY TRANSACTION_DATE DESC) AS rn
#   FROM cleaned_data
# )
# SELECT 
# POLICY_NUMBER,
# REMARKS,
# TRANSACTION_DATE,
# STATUS,
# SOURCE,
# BILLING_TYPE,
# CASE
#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'cybs' AND (
#     REMARKS IS NULL OR
#     LOWER(REMARKS) LIKE '%inactive card%' OR
#     LOWER(REMARKS) LIKE '%invalid account number%' OR
#     LOWER(REMARKS) LIKE '%stolen or lost card%' OR
#     LOWER(REMARKS) LIKE '%no such issuer%' OR
#     LOWER(REMARKS) LIKE '%invalid card brand' OR
#     LOWER(REMARKS) LIKE '%invalid card brand china union pay%' OR
#     LOWER(REMARKS) LIKE '%invalid card brand discover%' OR
#     LOWER(REMARKS) LIKE '%invalid card brand household%' OR
#     LOWER(REMARKS) LIKE '%blank token%' OR
#     LOWER(REMARKS) LIKE '%the issuing bank has questions%'
#   ) THEN 'Hard Decline'

#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'cybs' AND (
#     LOWER(REMARKS) LIKE '%api error%' OR
#     LOWER(REMARKS) LIKE '%err001%' OR
#     LOWER(REMARKS) LIKE '%err002%' OR
#     LOWER(REMARKS) LIKE '%expired card%' OR
#     LOWER(REMARKS) LIKE '%general decline%' OR
#     LOWER(REMARKS) LIKE '%insufficient funds%' OR
#     LOWER(REMARKS) LIKE '%invalid card verification number%' OR
#     LOWER(REMARKS) LIKE '%credit limit%' OR
#     LOWER(REMARKS) LIKE '%missing one or more fields%' OR
#     LOWER(REMARKS) LIKE '%invalid amt%' OR
#     LOWER(REMARKS) LIKE '%issuer inoperative%' OR
#     LOWER(REMARKS) LIKE '%no savings account%' OR
#     LOWER(REMARKS) LIKE '%system malfunction%' OR
#     LOWER(REMARKS) LIKE '%violation of law%' OR
#     LOWER(REMARKS) LIKE '%missing card brand%' OR
#     LOWER(REMARKS) LIKE '%reference number is missing%'
#   ) THEN 'Soft Decline'

#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'fiserv' THEN 'Soft Decline'
#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) IN ('ada', 'aca') THEN 'Others'
#   WHEN LOWER(STATUS) = 'declined' THEN 'Others'
#   ELSE 'Others'
#   END AS Decline_Category,
#   CARD_BRAND,
#   CARD_TYPE,
#   CARD_LEVEL,
#   BANK
# FROM label_latest_transaction
# WHERE rn = 1
# ),
# -- Clients Table
# clients AS (
# SELECT
#   clientID,
#   Salutation,
#   SurName,
#   FirstName,
#   Email,
#   CASE 
#     WHEN Email = '' THEN 0
#     WHEN Email IS NULL THEN 0
#     WHEN Email LIKE '%["(),:;<>\]%' THEN 0
#     WHEN SUBSTRING(Email, CHARINDEX('@', Email), LEN(Email)) LIKE '%[!#$%&*+/=?^`_{|]%' THEN 0
#     WHEN LEFT(Email, 1) LIKE '[-_.+]' OR RIGHT(Email, 1) LIKE '[-_.+]' THEN 0
#     WHEN Email LIKE '%[%' OR Email LIKE '%]%' THEN 0
#     WHEN Email LIKE '%@%@%' THEN 0
#     WHEN Email NOT LIKE '_%@_%._%' THEN 0
#     WHEN CHARINDEX('.', REVERSE(LTRIM(RTRIM(Email)))) < 2 THEN 0
#     WHEN CHARINDEX('.@', Email) <> 0 AND CHARINDEX('..', Email) <> 0 THEN 0
#     WHEN Email LIKE '%dummy@dummy.com%' THEN 0
#     WHEN Email LIKE '%PLUK%' OR Email LIKE '%pluk%' OR Email LIKE '%Pluk%' THEN 0
#     ELSE 1
#   END AS EmailValidityCheck,
# CASE 
#   WHEN MobileNo IS NULL OR TRIM(MobileNo) IN ('', '0', 'NA', '-') THEN NULL
#   ELSE MobileNo
# END AS MobileNo,  
# CASE 
#   WHEN RIGHT(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10) RLIKE '^[0-9]{10}$' AND RIGHT(LPAD(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10, '0'), 10) <> '0000000000' THEN RIGHT(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10)
#   ELSE '9898'
# END AS MobileNoCheck,
# CASE 
#   WHEN Telno1 IS NULL OR TRIM(Telno1) IN ('', '0', 'NA', '-') THEN NULL
#   ELSE Telno1
# END AS Telno1,
# CASE 
#   WHEN Telno2 IS NULL OR TRIM(Telno2) IN ('', '0', 'NA', '-') THEN NULL
#   ELSE Telno2
# END AS Telno2,
#   CASE 
#     WHEN (
#       (Email IS NOT NULL AND Email != '' AND
#         CASE 
#           WHEN Email LIKE '%["(),:;<>\]%' THEN 0
#           WHEN SUBSTRING(Email, CHARINDEX('@', Email), LEN(Email)) LIKE '%[!#$%&*+/=?^`_{|]%' THEN 0
#           WHEN LEFT(Email, 1) LIKE '[-_.+]' OR RIGHT(Email, 1) LIKE '[-_.+]' THEN 0
#           WHEN Email LIKE '%[%' OR Email LIKE '%]%' THEN 0
#           WHEN Email LIKE '%@%@%' THEN 0
#           WHEN Email NOT LIKE '_%@_%._%' THEN 0
#           WHEN CHARINDEX('.', REVERSE(LTRIM(RTRIM(Email)))) < 2 THEN 0
#           WHEN CHARINDEX('.@', Email) <> 0 AND CHARINDEX('..', Email) <> 0 THEN 0
#           WHEN Email LIKE '%dummy@dummy.com%' THEN 0
#           WHEN Email LIKE '%PLUK%' OR Email LIKE '%pluk%' OR Email LIKE '%Pluk%' THEN 0
#           ELSE 1
#         END = 1
#       )
#     OR (MobileNo IS NOT NULL AND TRIM(MobileNo) NOT IN ('', '0', 'NA', '-'))
#     OR (Telno1 IS NOT NULL AND TRIM(Telno1) NOT IN ('', '0', 'NA', '-'))
#     OR (Telno2 IS NOT NULL AND TRIM(Telno2) NOT IN ('', '0', 'NA', '-'))
#     ) THEN 1
#     ELSE 0
#   END AS HasValidContact
# FROM ext_scv_catalog.dbo.clients
# ),
# -- Previous Agents
# PREVAGENT AS (
#     WITH CTE AS (
#       SELECT 
#         PolicyNo,
#         AgentCode,
#         CurrentFrom,
#         ROW_NUMBER() OVER (PARTITION BY PolicyNo ORDER BY CurrentFrom DESC, COALESCE(CurrentTo, TIMESTAMP('2070-12-31 00:00:00')) DESC) AS rn
#       FROM ext_scv_catalog.dbo.policyagenthistory
#       WHERE AgentCode NOT IN ('70007328', '70009250')
#     )
#     SELECT 
#       a.PolicyNo,
#       a.AgentCode,
#       b.ClientID,
#       b.Surname,
#       b.FirstName
#     FROM CTE a
#     LEFT JOIN (
#       SELECT 
#         ag.AgentCode, 
#         ag.ClientID,
#         cl.SurName, 
#         cl.FirstName
#       FROM ext_scv_catalog.dbo.agents ag
#       LEFT JOIN clients cl 
#       ON ag.ClientID = cl.ClientID
#     ) b
#     ON a.AgentCode = b.AgentCode
#     WHERE a.rn = 1
# ),
# -- vwcontracts Table
# vwcontracts AS (
# SELECT
# DISTINCT
# PolicyNo,
# owner,
# CAST(FirstRCD AS DATE) AS FirstRDC,
# CASE
#   WHEN ContractType IN ('YP5', 'YP7', 'YPT', 'YPF', 'YD5', 'YD7', 'YDT', 'YDF', 'XP5', 'XP7', 'XPT', 'XPF', 'XLT', 'XLF', 'XD5', 'XD7', 'XDT', 'XDF', 'R05', 'R07', 'R10', 'R15', 'N05', 'N07', 'N10', 'N15', 'D05', 'D07', 'D10', 'D15', 'C20', 'RLP', 'RDP', 'RPA', 'RPB', 'RPD', 'PHP', 'PHD', 'PIA', 'PI2', 'P3P', 'P3D', 'P3L', 'P3M', 'RPW', 'RW1', 'RW2', 'CAP', 'YP2') 
#   THEN 'UL'
#   WHEN ContractType IN ('E20', 'AAA', 'PRO', 'PLP', 'BCB', 'BCE', 'EWR', 'CB1', 'CB2', 'CB3', 'CB4', 'CB5', 'ADP', 'DHI', 'TLR', 'ACP', 'LCP', 'MCP', 'PPA', 'LPB', 'LPC', 'LPD', 'LPA', 'W5P', 'W6P', 'PPP', 'WPP', 'LH1', 'LH2', 'LH3', 'LH4', 'PCA', 'D12', 'D06', 'P12', 'P06', 'PDL', 'E2S', 'PFL', 'PLD', 'PLL', 'P5L', 'PHA', 'HG1', 'HG2', 'HG3', 'PLB', 'PPL', 'W2P', 'WPL', 'W4P', 'I12', 'I06', 'P1S', 'P2S', 'W2A', 'P3S', 'PS0', 'PS2', 'PS3', 'PS4', 'PS6', 'PS7', 'PS8', 'PS1', 'PS5', 'PS9', 'P5P', 'PP5', 'W1P', 'WP5', 'PW1', 'PW2', 'PW3', 'PW4', 'RRI', 'CTM', 'STM', 'HTM', 'CT3', 'CTR', 'T15', 'T90', 'WLR', 'WLU', 'SPA', 'LI5', 'LIT')
#   THEN 'Trad'
# END AS ProductGroup,
# CAST(PaidToDate AS DATE) AS PaidToDate,
# DATEDIFF(MONTH, PaidToDate, CURRENT_DATE()) AS Datedif,
# CAST(BillToDate AS DATE) AS BillToDate,
# CASE
#   WHEN BillFreq = 1 THEN 'Annual'
#   WHEN BillFreq = 2 THEN 'Semi-annual'
#   WHEN BillFreq = 4 THEN 'Quarterly'
#   WHEN BillFreq = 12 THEN 'Monthly'
#   ELSE 'Single Pay'
# END AS BillFreq,
# InstPrem * BillFreq AS APE,
# CASE
#   WHEN PayMode = 'C' THEN 'Cash'
#   WHEN PayMode = 'D' THEN 'Auto-debit'
#   WHEN PayMode = 'K' THEN 'Auto-credit'
#   WHEN PayMode = 'T' THEN 'Auto-pay'
#   WHEN PayMode = 'G' THEN 'Group'
#   ELSE 'Others'
# END AS PayMode,
# Currency,
# InstPrem,
# RiskStatus,
# PremStatus,
# COUNT(PolicyNo) OVER (PARTITION BY Owner) AS PolicyCount,
# AgentCode,
# ContractType
# FROM ext_scv_catalog.dbo.vwcontracts
# WHERE Owner NOT IN (
#   SELECT
#   Owner
#   FROM ext_scv_catalog.dbo.vwcontracts
#   WHERE PolicyNo IN (
#   SELECT
#   Policy_Number
#   FROM curated_catalog.analytics.call_center_dnc
#   )
# )
# ),
# -- Contract Type
# contracttype AS (
# SELECT
# TRIM(ContractType) AS ContractType,
# TRIM(ConTypeShortDesc) AS ConTypeShortDesc,
# TRIM(ConTypeLongDesc) AS ConTypeLongDesc
# FROM ext_scv_catalog.dbo.contracttype
# ),
# -- Orphan Accounts
# orphan AS (
# SELECT 
# TRIM(REPLACE(REPLACE(REPLACE(CHDRNUM, ',' , ''), ' ', ''), '.', '')) AS POLICY_NUMBER
# FROM ds_catalog.str_lifeasia.zchdpf
# WHERE isValid = 1 AND ORPTAG = 'ORP' 
# ),
# -- Latest Transaction -> Total Count: 745544
# -- Main Table - Auto Decline Data Extraction
# autodeclined AS (
# SELECT
# alt.POLICY_NUMBER,
# CASE
#   WHEN vc.PolicyCount > 1 THEN 'Yes'
#   ELSE 'No'
# END AS MultiplePolicies,
# vc.Owner,
# cl.Salutation,
# cl.SurName,
# cl.FirstName,
# vc.FirstRDC,
# ct.ConTypeLongDesc,
# vc.ProductGroup,
# CASE
#   WHEN vc.AgentCode IN ('70007328', '70009250') THEN pa.AgentCode
#   ELSE vc.AgentCode
# END AS AgentCode,
# CASE
#   WHEN vc.AgentCode IN ('70007328', '70009250') THEN CONCAT(pa.Surname, ', ', pa.FirstName)
#   ELSE CONCAT(ag.Surname, ' ', ag.FirstName)
# END AS AgentName,
# CASE
#   WHEN o.POLICY_NUMBER IS NOT NULL THEN 'Yes'
#   ELSE 'No'
# END AS OrphanAccount,
# vc.PaidToDate,
# vc.DateDif,
# vc.Currency,
# vc.InstPrem,
# vc.BillFreq,
# vc.APE,
# vc.PayMode,
# cl.Email,
# cl.EmailValidityCheck,
# cl.HasValidContact,
# cl.MobileNo,
# cl.MobileNoCheck,
# cl.Telno1,
# cl.Telno2,
# vc.RiskStatus,
# vc.PremStatus,
# CASE
#   WHEN cl.Salutation IS NULL OR cl.Salutation = '' OR vc.PayMode = 'G' THEN 'Corporate'
#   ELSE 'Non-corporate'
# END AS ClientType,
# alt.REMARKS,
# alt.Decline_Category,
# alt.TRANSACTION_DATE AS LatestTransactionDate,
# alt.STATUS,
# alt.BILLING_TYPE,
# alt.CARD_BRAND,
# alt.CARD_TYPE,
# alt.CARD_LEVEL,
# alt.BANK,
# alt.SOURCE
# FROM autopay_latesttransaction alt
# LEFT JOIN vwcontracts vc
# ON alt.POLICY_NUMBER = vc.PolicyNo
# LEFT JOIN clients cl
# ON vc.Owner = cl.ClientID
# LEFT JOIN (
#   SELECT
#   ag.AgentCode,
#   ag.ClientID,
#   cl.SurName,
#   cl.FirstName
#   FROM ext_scv_catalog.dbo.agents ag
#   LEFT JOIN clients cl
#   ON ag.ClientID = cl.ClientID
# ) ag
# ON vc.AgentCode = ag.AgentCode
# LEFT JOIN contracttype ct
# ON vc.ContractType = ct.ContractType
# LEFT JOIN PREVAGENT pa
# ON vc.PolicyNo = pa.PolicyNo
# LEFT JOIN orphan o
# ON alt.POLICY_NUMBER = o.POLICY_NUMBER
# )
# SELECT *
# FROM autodeclined

In [0]:
# %sql
# WITH autopay_latesttransaction AS (
# WITH cleaned_data AS (
#   SELECT 
#     TRIM(REPLACE(REPLACE(REPLACE(POLICY_NUMBER, ',' , ''), ' ', ''), '.', '')) AS POLICY_NUMBER,
#     REMARKS,
#     TRANSACTION_DATE,
#     STATUS,
#     SOURCE,
#     BILLING_TYPE,
#     CARD_BRAND,
#     CARD_TYPE,
#     CARD_LEVEL,
#     BANK
#   FROM curated_catalog.analytics.autopaybilling_daily
# ),
# label_latest_transaction AS (
#   SELECT 
#     POLICY_NUMBER AS POLICY_NUMBER,
#     REMARKS,
#     TRANSACTION_DATE,
#     STATUS,
#     SOURCE,
#     BILLING_TYPE,
#     CARD_BRAND,
#     CARD_TYPE,
#     CARD_LEVEL,
#     BANK,
#     ROW_NUMBER() OVER (PARTITION BY POLICY_NUMBER ORDER BY TRANSACTION_DATE DESC) AS rn
#   FROM cleaned_data
# )
# SELECT 
# POLICY_NUMBER,
# REMARKS,
# TRANSACTION_DATE,
# STATUS,
# SOURCE,
# BILLING_TYPE,
# CASE
#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'cybs' AND (
#     REMARKS IS NULL OR
#     LOWER(REMARKS) LIKE '%inactive card%' OR
#     LOWER(REMARKS) LIKE '%invalid account number%' OR
#     LOWER(REMARKS) LIKE '%stolen or lost card%' OR
#     LOWER(REMARKS) LIKE '%no such issuer%' OR
#     LOWER(REMARKS) LIKE '%invalid card brand' OR
#     LOWER(REMARKS) LIKE '%invalid card brand china union pay%' OR
#     LOWER(REMARKS) LIKE '%invalid card brand discover%' OR
#     LOWER(REMARKS) LIKE '%invalid card brand household%' OR
#     LOWER(REMARKS) LIKE '%blank token%' OR
#     LOWER(REMARKS) LIKE '%the issuing bank has questions%'
#   ) THEN 'Hard Decline'

#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'cybs' AND (
#     LOWER(REMARKS) LIKE '%api error%' OR
#     LOWER(REMARKS) LIKE '%err001%' OR
#     LOWER(REMARKS) LIKE '%err002%' OR
#     LOWER(REMARKS) LIKE '%expired card%' OR
#     LOWER(REMARKS) LIKE '%general decline%' OR
#     LOWER(REMARKS) LIKE '%insufficient funds%' OR
#     LOWER(REMARKS) LIKE '%invalid card verification number%' OR
#     LOWER(REMARKS) LIKE '%credit limit%' OR
#     LOWER(REMARKS) LIKE '%missing one or more fields%' OR
#     LOWER(REMARKS) LIKE '%invalid amt%' OR
#     LOWER(REMARKS) LIKE '%issuer inoperative%' OR
#     LOWER(REMARKS) LIKE '%no savings account%' OR
#     LOWER(REMARKS) LIKE '%system malfunction%' OR
#     LOWER(REMARKS) LIKE '%violation of law%' OR
#     LOWER(REMARKS) LIKE '%missing card brand%' OR
#     LOWER(REMARKS) LIKE '%reference number is missing%'
#   ) THEN 'Soft Decline'

#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'fiserv' THEN 'Soft Decline'
#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) IN ('ada', 'aca') THEN 'Others'
#   WHEN LOWER(STATUS) = 'declined' THEN 'Others'
#   ELSE 'Others'
#   END AS Decline_Category,
#   CARD_BRAND,
#   CARD_TYPE,
#   CARD_LEVEL,
#   BANK
# FROM label_latest_transaction
# WHERE rn = 1
# ),
# -- Clients Table
# clients AS (
# SELECT
#   clientID,
#   Salutation,
#   SurName,
#   FirstName,
#   Email,
#   CASE 
#     WHEN Email = '' THEN 0
#     WHEN Email IS NULL THEN 0
#     WHEN Email LIKE '%["(),:;<>\]%' THEN 0
#     WHEN SUBSTRING(Email, CHARINDEX('@', Email), LEN(Email)) LIKE '%[!#$%&*+/=?^`_{|]%' THEN 0
#     WHEN LEFT(Email, 1) LIKE '[-_.+]' OR RIGHT(Email, 1) LIKE '[-_.+]' THEN 0
#     WHEN Email LIKE '%[%' OR Email LIKE '%]%' THEN 0
#     WHEN Email LIKE '%@%@%' THEN 0
#     WHEN Email NOT LIKE '_%@_%._%' THEN 0
#     WHEN CHARINDEX('.', REVERSE(LTRIM(RTRIM(Email)))) < 2 THEN 0
#     WHEN CHARINDEX('.@', Email) <> 0 AND CHARINDEX('..', Email) <> 0 THEN 0
#     WHEN Email LIKE '%dummy@dummy.com%' THEN 0
#     WHEN Email LIKE '%PLUK%' OR Email LIKE '%pluk%' OR Email LIKE '%Pluk%' THEN 0
#     ELSE 1
#   END AS EmailValidityCheck,
# CASE 
#   WHEN MobileNo IS NULL OR TRIM(MobileNo) IN ('', '0', 'NA', '-') THEN NULL
#   ELSE MobileNo
# END AS MobileNo,  
# CASE 
#   WHEN RIGHT(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10) RLIKE '^[0-9]{10}$' AND RIGHT(LPAD(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10, '0'), 10) <> '0000000000' THEN RIGHT(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10)
#   ELSE '9898'
# END AS MobileNoCheck,
# CASE 
#   WHEN Telno1 IS NULL OR TRIM(Telno1) IN ('', '0', 'NA', '-') THEN NULL
#   ELSE Telno1
# END AS Telno1,
# CASE 
#   WHEN Telno2 IS NULL OR TRIM(Telno2) IN ('', '0', 'NA', '-') THEN NULL
#   ELSE Telno2
# END AS Telno2,
#   CASE 
#     WHEN (
#       (Email IS NOT NULL AND Email != '' AND
#         CASE 
#           WHEN Email LIKE '%["(),:;<>\]%' THEN 0
#           WHEN SUBSTRING(Email, CHARINDEX('@', Email), LEN(Email)) LIKE '%[!#$%&*+/=?^`_{|]%' THEN 0
#           WHEN LEFT(Email, 1) LIKE '[-_.+]' OR RIGHT(Email, 1) LIKE '[-_.+]' THEN 0
#           WHEN Email LIKE '%[%' OR Email LIKE '%]%' THEN 0
#           WHEN Email LIKE '%@%@%' THEN 0
#           WHEN Email NOT LIKE '_%@_%._%' THEN 0
#           WHEN CHARINDEX('.', REVERSE(LTRIM(RTRIM(Email)))) < 2 THEN 0
#           WHEN CHARINDEX('.@', Email) <> 0 AND CHARINDEX('..', Email) <> 0 THEN 0
#           WHEN Email LIKE '%dummy@dummy.com%' THEN 0
#           WHEN Email LIKE '%PLUK%' OR Email LIKE '%pluk%' OR Email LIKE '%Pluk%' THEN 0
#           ELSE 1
#         END = 1
#       )
#     OR (MobileNo IS NOT NULL AND TRIM(MobileNo) NOT IN ('', '0', 'NA', '-'))
#     OR (Telno1 IS NOT NULL AND TRIM(Telno1) NOT IN ('', '0', 'NA', '-'))
#     OR (Telno2 IS NOT NULL AND TRIM(Telno2) NOT IN ('', '0', 'NA', '-'))
#     ) THEN 1
#     ELSE 0
#   END AS HasValidContact
# FROM ext_scv_catalog.dbo.clients
# ),
# -- Previous Agents
# PREVAGENT AS (
#     WITH CTE AS (
#       SELECT 
#         PolicyNo,
#         AgentCode,
#         CurrentFrom,
#         ROW_NUMBER() OVER (PARTITION BY PolicyNo ORDER BY CurrentFrom DESC, COALESCE(CurrentTo, TIMESTAMP('2070-12-31 00:00:00')) DESC) AS rn
#       FROM ext_scv_catalog.dbo.policyagenthistory
#       WHERE AgentCode NOT IN ('70007328', '70009250')
#     )
#     SELECT 
#       a.PolicyNo,
#       a.AgentCode,
#       b.ClientID,
#       b.Surname,
#       b.FirstName
#     FROM CTE a
#     LEFT JOIN (
#       SELECT 
#         ag.AgentCode, 
#         ag.ClientID,
#         cl.SurName, 
#         cl.FirstName
#       FROM ext_scv_catalog.dbo.agents ag
#       LEFT JOIN clients cl 
#       ON ag.ClientID = cl.ClientID
#     ) b
#     ON a.AgentCode = b.AgentCode
#     WHERE a.rn = 1
# ),
# -- vwcontracts Table
# vwcontracts AS (
# SELECT
# DISTINCT
# PolicyNo,
# owner,
# CAST(FirstRCD AS DATE) AS FirstRDC,
# CASE
#   WHEN ContractType IN ('YP5', 'YP7', 'YPT', 'YPF', 'YD5', 'YD7', 'YDT', 'YDF', 'XP5', 'XP7', 'XPT', 'XPF', 'XLT', 'XLF', 'XD5', 'XD7', 'XDT', 'XDF', 'R05', 'R07', 'R10', 'R15', 'N05', 'N07', 'N10', 'N15', 'D05', 'D07', 'D10', 'D15', 'C20', 'RLP', 'RDP', 'RPA', 'RPB', 'RPD', 'PHP', 'PHD', 'PIA', 'PI2', 'P3P', 'P3D', 'P3L', 'P3M', 'RPW', 'RW1', 'RW2', 'CAP', 'YP2') 
#   THEN 'UL'
#   WHEN ContractType IN ('E20', 'AAA', 'PRO', 'PLP', 'BCB', 'BCE', 'EWR', 'CB1', 'CB2', 'CB3', 'CB4', 'CB5', 'ADP', 'DHI', 'TLR', 'ACP', 'LCP', 'MCP', 'PPA', 'LPB', 'LPC', 'LPD', 'LPA', 'W5P', 'W6P', 'PPP', 'WPP', 'LH1', 'LH2', 'LH3', 'LH4', 'PCA', 'D12', 'D06', 'P12', 'P06', 'PDL', 'E2S', 'PFL', 'PLD', 'PLL', 'P5L', 'PHA', 'HG1', 'HG2', 'HG3', 'PLB', 'PPL', 'W2P', 'WPL', 'W4P', 'I12', 'I06', 'P1S', 'P2S', 'W2A', 'P3S', 'PS0', 'PS2', 'PS3', 'PS4', 'PS6', 'PS7', 'PS8', 'PS1', 'PS5', 'PS9', 'P5P', 'PP5', 'W1P', 'WP5', 'PW1', 'PW2', 'PW3', 'PW4', 'RRI', 'CTM', 'STM', 'HTM', 'CT3', 'CTR', 'T15', 'T90', 'WLR', 'WLU', 'SPA', 'LI5', 'LIT')
#   THEN 'Trad'
# END AS ProductGroup,
# CAST(PaidToDate AS DATE) AS PaidToDate,
# DATEDIFF(MONTH, PaidToDate, CURRENT_DATE()) AS Datedif,
# CAST(BillToDate AS DATE) AS BillToDate,
# CASE
#   WHEN BillFreq = 1 THEN 'Annual'
#   WHEN BillFreq = 2 THEN 'Semi-annual'
#   WHEN BillFreq = 4 THEN 'Quarterly'
#   WHEN BillFreq = 12 THEN 'Monthly'
#   ELSE 'Single Pay'
# END AS BillFreq,
# InstPrem * BillFreq AS APE,
# CASE
#   WHEN PayMode = 'C' THEN 'Cash'
#   WHEN PayMode = 'D' THEN 'Auto-debit'
#   WHEN PayMode = 'K' THEN 'Auto-credit'
#   WHEN PayMode = 'T' THEN 'Auto-pay'
#   WHEN PayMode = 'G' THEN 'Group'
#   ELSE 'Others'
# END AS PayMode,
# Currency,
# InstPrem,
# RiskStatus,
# PremStatus,
# COUNT(PolicyNo) OVER (PARTITION BY Owner) AS PolicyCount,
# AgentCode,
# ContractType
# FROM ext_scv_catalog.dbo.vwcontracts
# ),
# contracttype AS (
# SELECT
# TRIM(ContractType) AS ContractType,
# TRIM(ConTypeShortDesc) AS ConTypeShortDesc,
# TRIM(ConTypeLongDesc) AS ConTypeLongDesc
# FROM ext_scv_catalog.dbo.contracttype
# ),
# -- Orphan Accounts
# orphan AS (
# SELECT 
# TRIM(REPLACE(REPLACE(REPLACE(CHDRNUM, ',' , ''), ' ', ''), '.', '')) AS POLICY_NUMBER
# FROM ds_catalog.str_lifeasia.zchdpf
# WHERE isValid = 1 AND ORPTAG = 'ORP' 
# ),
# -- Latest Transaction -> Total Count: 745544
# -- Main Table - Auto Decline Data Extraction
# autodeclined AS (
# SELECT
# alt.POLICY_NUMBER,
# CASE
#   WHEN vc.PolicyCount > 1 THEN 'Yes'
#   ELSE 'No'
# END AS MultiplePolicies,
# vc.Owner,
# cl.Salutation,
# cl.SurName,
# cl.FirstName,
# vc.FirstRDC,
# ct.ConTypeLongDesc,
# vc.ProductGroup,
# CASE
#   WHEN vc.AgentCode IN ('70007328', '70009250') THEN pa.AgentCode
#   ELSE vc.AgentCode
# END AS AgentCode,
# CASE
#   WHEN vc.AgentCode IN ('70007328', '70009250') THEN CONCAT(pa.Surname, ', ', pa.FirstName)
#   ELSE CONCAT(ag.Surname, ' ', ag.FirstName)
# END AS AgentName,
# CASE
#   WHEN o.POLICY_NUMBER IS NOT NULL THEN 'Yes'
#   ELSE 'No'
# END AS OrphanAccount,
# vc.PaidToDate,
# vc.DateDif,
# vc.Currency,
# vc.InstPrem,
# vc.BillFreq,
# vc.APE,
# vc.PayMode,
# cl.Email,
# cl.EmailValidityCheck,
# cl.HasValidContact,
# cl.MobileNo,
# cl.MobileNoCheck,
# cl.Telno1,
# cl.Telno2,
# vc.RiskStatus,
# vc.PremStatus,
# CASE
#   WHEN cl.Salutation IS NULL OR cl.Salutation = '' OR vc.PayMode = 'G' THEN 'Corporate'
#   ELSE 'Non-corporate'
# END AS ClientType,
# alt.REMARKS,
# alt.Decline_Category,
# alt.TRANSACTION_DATE AS LatestTransactionDate,
# alt.STATUS,
# alt.BILLING_TYPE,
# alt.CARD_BRAND,
# alt.CARD_TYPE,
# alt.CARD_LEVEL,
# alt.BANK,
# alt.SOURCE
# FROM autopay_latesttransaction alt
# LEFT JOIN vwcontracts vc
# ON alt.POLICY_NUMBER = vc.PolicyNo
# LEFT JOIN clients cl
# ON vc.Owner = cl.ClientID
# LEFT JOIN (
#   SELECT
#   ag.AgentCode,
#   ag.ClientID,
#   cl.SurName,
#   cl.FirstName
#   FROM ext_scv_catalog.dbo.agents ag
#   LEFT JOIN clients cl
#   ON ag.ClientID = cl.ClientID
# ) ag
# ON vc.AgentCode = ag.AgentCode
# LEFT JOIN contracttype ct
# ON vc.ContractType = ct.ContractType
# LEFT JOIN PREVAGENT pa
# ON vc.PolicyNo = pa.PolicyNo
# LEFT JOIN orphan o
# ON alt.POLICY_NUMBER = o.POLICY_NUMBER
# )
# SELECT *
# FROM autodeclined

In [0]:
# %sql
# WITH autopay_latesttransaction AS (
# WITH cleaned_data AS (
#   SELECT 
#     TRIM(REPLACE(REPLACE(REPLACE(POLICY_NUMBER, ',' , ''), ' ', ''), '.', '')) AS POLICY_NUMBER,
#     REMARKS,
#     TRANSACTION_DATE,
#     PAID_TO_DATE,
#     STATUS,
#     SOURCE,
#     BILLING_TYPE
#   FROM curated_catalog.analytics.autopaybilling_daily
# ),
# label_latest_transaction AS (
#   SELECT 
#     POLICY_NUMBER AS POLICY_NUMBER,
#     REMARKS,
#     TRANSACTION_DATE,
#     PAID_TO_DATE,
#     STATUS,
#     SOURCE,
#     BILLING_TYPE,
#     ROW_NUMBER() OVER (PARTITION BY POLICY_NUMBER ORDER BY TRANSACTION_DATE DESC) AS rn
#   FROM cleaned_data
# )
# SELECT 
# POLICY_NUMBER,
# REMARKS,
# TRANSACTION_DATE,
# PAID_TO_DATE,
# STATUS,
# SOURCE,
# BILLING_TYPE,
# CASE
#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'cybs' AND (
#     REMARKS IS NULL OR
#     LOWER(REMARKS) LIKE '%inactive card%' OR
#     LOWER(REMARKS) LIKE '%invalid account number%' OR
#     LOWER(REMARKS) LIKE '%stolen or lost card%' OR
#     LOWER(REMARKS) LIKE '%no such issuer%' OR
#     LOWER(REMARKS) LIKE '%invalid card brand' OR
#     LOWER(REMARKS) LIKE '%invalid card brand china union pay%' OR
#     LOWER(REMARKS) LIKE '%invalid card brand discover%' OR
#     LOWER(REMARKS) LIKE '%invalid card brand household%' OR
#     LOWER(REMARKS) LIKE '%blank token%' OR
#     LOWER(REMARKS) LIKE '%the issuing bank has questions%'
#   ) THEN 'Hard Decline'

#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'cybs' AND (
#     LOWER(REMARKS) LIKE '%api error%' OR
#     LOWER(REMARKS) LIKE '%err001%' OR
#     LOWER(REMARKS) LIKE '%err002%' OR
#     LOWER(REMARKS) LIKE '%expired card%' OR
#     LOWER(REMARKS) LIKE '%general decline%' OR
#     LOWER(REMARKS) LIKE '%insufficient funds%' OR
#     LOWER(REMARKS) LIKE '%invalid card verification number%' OR
#     LOWER(REMARKS) LIKE '%credit limit%' OR
#     LOWER(REMARKS) LIKE '%missing one or more fields%' OR
#     LOWER(REMARKS) LIKE '%invalid amt%' OR
#     LOWER(REMARKS) LIKE '%issuer inoperative%' OR
#     LOWER(REMARKS) LIKE '%no savings account%' OR
#     LOWER(REMARKS) LIKE '%system malfunction%' OR
#     LOWER(REMARKS) LIKE '%violation of law%' OR
#     LOWER(REMARKS) LIKE '%missing card brand%' OR
#     LOWER(REMARKS) LIKE '%reference number is missing%'
#   ) THEN 'Soft Decline'

#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'fiserv' THEN 'Soft Decline'
#   WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) IN ('ada', 'aca') THEN 'Others'
#   WHEN LOWER(STATUS) = 'declined' THEN 'Others'
#   ELSE 'Others'
#   END AS Decline_Category
# FROM label_latest_transaction
# WHERE rn = 1
# ),
# -- Clients Table
# clients AS (
# SELECT
#   clientID,
#   Salutation,
#   SurName,
#   FirstName,
#   Email,
#   CASE 
#     WHEN Email = '' THEN 0
#     WHEN Email IS NULL THEN 0
#     WHEN Email LIKE '%["(),:;<>\]%' THEN 0
#     WHEN SUBSTRING(Email, CHARINDEX('@', Email), LEN(Email)) LIKE '%[!#$%&*+/=?^`_{|]%' THEN 0
#     WHEN LEFT(Email, 1) LIKE '[-_.+]' OR RIGHT(Email, 1) LIKE '[-_.+]' THEN 0
#     WHEN Email LIKE '%[%' OR Email LIKE '%]%' THEN 0
#     WHEN Email LIKE '%@%@%' THEN 0
#     WHEN Email NOT LIKE '_%@_%._%' THEN 0
#     WHEN CHARINDEX('.', REVERSE(LTRIM(RTRIM(Email)))) < 2 THEN 0
#     WHEN CHARINDEX('.@', Email) <> 0 AND CHARINDEX('..', Email) <> 0 THEN 0
#     WHEN Email LIKE '%dummy@dummy.com%' THEN 0
#     WHEN Email LIKE '%PLUK%' OR Email LIKE '%pluk%' OR Email LIKE '%Pluk%' THEN 0
#     ELSE 1
#   END AS EmailValidityCheck,
# CASE 
#   WHEN MobileNo IS NULL OR TRIM(MobileNo) IN ('', '0', 'NA', '-') THEN NULL
#   ELSE MobileNo
# END AS MobileNo,  
# CASE 
#   WHEN RIGHT(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10) RLIKE '^[0-9]{10}$' AND RIGHT(LPAD(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10, '0'), 10) <> '0000000000' THEN RIGHT(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10)
#   ELSE '9898'
# END AS MobileNoCheck,
# CASE 
#   WHEN Telno1 IS NULL OR TRIM(Telno1) IN ('', '0', 'NA', '-') THEN NULL
#   ELSE Telno1
# END AS Telno1,
# CASE 
#   WHEN Telno2 IS NULL OR TRIM(Telno2) IN ('', '0', 'NA', '-') THEN NULL
#   ELSE Telno2
# END AS Telno2,
#   CASE 
#     WHEN (
#       (Email IS NOT NULL AND Email != '' AND
#         CASE 
#           WHEN Email LIKE '%["(),:;<>\]%' THEN 0
#           WHEN SUBSTRING(Email, CHARINDEX('@', Email), LEN(Email)) LIKE '%[!#$%&*+/=?^`_{|]%' THEN 0
#           WHEN LEFT(Email, 1) LIKE '[-_.+]' OR RIGHT(Email, 1) LIKE '[-_.+]' THEN 0
#           WHEN Email LIKE '%[%' OR Email LIKE '%]%' THEN 0
#           WHEN Email LIKE '%@%@%' THEN 0
#           WHEN Email NOT LIKE '_%@_%._%' THEN 0
#           WHEN CHARINDEX('.', REVERSE(LTRIM(RTRIM(Email)))) < 2 THEN 0
#           WHEN CHARINDEX('.@', Email) <> 0 AND CHARINDEX('..', Email) <> 0 THEN 0
#           WHEN Email LIKE '%dummy@dummy.com%' THEN 0
#           WHEN Email LIKE '%PLUK%' OR Email LIKE '%pluk%' OR Email LIKE '%Pluk%' THEN 0
#           ELSE 1
#         END = 1
#       )
#     OR (MobileNo IS NOT NULL AND TRIM(MobileNo) NOT IN ('', '0', 'NA', '-'))
#     OR (Telno1 IS NOT NULL AND TRIM(Telno1) NOT IN ('', '0', 'NA', '-'))
#     OR (Telno2 IS NOT NULL AND TRIM(Telno2) NOT IN ('', '0', 'NA', '-'))
#     ) THEN 1
#     ELSE 0
#   END AS HasValidContact
# FROM ext_scv_catalog.dbo.clients
# ),
# -- Previous Agents
# PREVAGENT AS (
#     WITH CTE AS (
#       SELECT 
#         PolicyNo,
#         AgentCode,
#         CurrentFrom,
#         ROW_NUMBER() OVER (PARTITION BY PolicyNo ORDER BY CurrentFrom DESC, COALESCE(CurrentTo, TIMESTAMP('2070-12-31 00:00:00')) DESC) AS rn
#       FROM ext_scv_catalog.dbo.policyagenthistory
#       WHERE AgentCode NOT IN ('70007328', '70009250')
#     )
#     SELECT 
#       a.PolicyNo,
#       a.AgentCode,
#       b.ClientID,
#       b.Surname,
#       b.FirstName
#     FROM CTE a
#     LEFT JOIN (
#       SELECT 
#         ag.AgentCode, 
#         ag.ClientID,
#         cl.SurName, 
#         cl.FirstName
#       FROM ext_scv_catalog.dbo.agents ag
#       LEFT JOIN clients cl 
#       ON ag.ClientID = cl.ClientID
#     ) b
#     ON a.AgentCode = b.AgentCode
#     WHERE a.rn = 1
# ),
# -- vwcontracts Table
# vwcontracts AS (
# SELECT
# DISTINCT
# PolicyNo,
# owner,
# CAST(FirstRCD AS DATE) AS FirstRDC,
# CASE
#   WHEN ContractType IN ('YP5', 'YP7', 'YPT', 'YPF', 'YD5', 'YD7', 'YDT', 'YDF', 'XP5', 'XP7', 'XPT', 'XPF', 'XLT', 'XLF', 'XD5', 'XD7', 'XDT', 'XDF', 'R05', 'R07', 'R10', 'R15', 'N05', 'N07', 'N10', 'N15', 'D05', 'D07', 'D10', 'D15', 'C20', 'RLP', 'RDP', 'RPA', 'RPB', 'RPD', 'PHP', 'PHD', 'PIA', 'PI2', 'P3P', 'P3D', 'P3L', 'P3M', 'RPW', 'RW1', 'RW2', 'CAP', 'YP2') 
#   THEN 'UL'
#   WHEN ContractType IN ('E20', 'AAA', 'PRO', 'PLP', 'BCB', 'BCE', 'EWR', 'CB1', 'CB2', 'CB3', 'CB4', 'CB5', 'ADP', 'DHI', 'TLR', 'ACP', 'LCP', 'MCP', 'PPA', 'LPB', 'LPC', 'LPD', 'LPA', 'W5P', 'W6P', 'PPP', 'WPP', 'LH1', 'LH2', 'LH3', 'LH4', 'PCA', 'D12', 'D06', 'P12', 'P06', 'PDL', 'E2S', 'PFL', 'PLD', 'PLL', 'P5L', 'PHA', 'HG1', 'HG2', 'HG3', 'PLB', 'PPL', 'W2P', 'WPL', 'W4P', 'I12', 'I06', 'P1S', 'P2S', 'W2A', 'P3S', 'PS0', 'PS2', 'PS3', 'PS4', 'PS6', 'PS7', 'PS8', 'PS1', 'PS5', 'PS9', 'P5P', 'PP5', 'W1P', 'WP5', 'PW1', 'PW2', 'PW3', 'PW4', 'RRI', 'CTM', 'STM', 'HTM', 'CT3', 'CTR', 'T15', 'T90', 'WLR', 'WLU', 'SPA', 'LI5', 'LIT')
#   THEN 'Trad'
# END AS ProductGroup,
# CAST(PaidToDate AS DATE) AS PaidToDate,
# DATEDIFF(MONTH, PaidToDate, CURRENT_DATE()) AS Datedif,
# CAST(BillToDate AS DATE) AS BillToDate,
# CASE
#   WHEN BillFreq = 1 THEN 'Annual'
#   WHEN BillFreq = 2 THEN 'Semi-annual'
#   WHEN BillFreq = 4 THEN 'Quarterly'
#   WHEN BillFreq = 12 THEN 'Monthly'
#   ELSE 'Single Pay'
# END AS BillFreq,
# Currency,
# InstPrem,
# InstPrem * BillFreq AS APE,
# CASE
#   WHEN PayMode = 'C' THEN 'Cash'
#   WHEN PayMode = 'D' THEN 'Auto-debit'
#   WHEN PayMode = 'K' THEN 'Auto-credit'
#   WHEN PayMode = 'T' THEN 'Auto-pay'
#   WHEN PayMode = 'G' THEN 'Group'
#   ELSE 'Others'
# END AS PayMode,
# InstPrem,
# RiskStatus,
# PremStatus,
# COUNT(PolicyNo) OVER (PARTITION BY Owner) AS PolicyCount,
# AgentCode,
# ContractType,
# BranchName
# FROM ext_scv_catalog.dbo.vwcontracts
# WHERE Owner NOT IN (
#   SELECT
#   Owner
#   FROM ext_scv_catalog.dbo.vwcontracts
#   WHERE PolicyNo IN (
#   SELECT
#   Policy_Number
#   FROM curated_catalog.analytics.call_center_dnc
#   )
# )
# ),
# contracttype AS (
# SELECT
# TRIM(ContractType) AS ContractType,
# TRIM(ConTypeShortDesc) AS ConTypeShortDesc,
# TRIM(ConTypeLongDesc) AS ConTypeLongDesc
# FROM ext_scv_catalog.dbo.contracttype
# ),
# -- Orphan Accounts
# orphan AS (
# SELECT 
# TRIM(REPLACE(REPLACE(REPLACE(CHDRNUM, ',' , ''), ' ', ''), '.', '')) AS POLICY_NUMBER
# FROM ds_catalog.str_lifeasia.zchdpf
# WHERE isValid = 1 AND ORPTAG = 'ORP' 
# ),
# -- Latest Transaction -> Total Count: 745544
# -- Main Table - Auto Decline Data Extraction
# autodeclined AS (
# SELECT
# alt.POLICY_NUMBER,
# CASE
#   WHEN vc.PolicyCount > 1 THEN 'Yes'
#   ELSE 'No'
# END AS MultiplePolicies,
# vc.Owner,
# cl.Salutation,
# cl.SurName,
# cl.FirstName,
# vc.FirstRDC,
# ct.ConTypeLongDesc,
# vc.ProductGroup,
# CASE
#   WHEN vc.AgentCode IN ('70007328', '70009250') THEN pa.AgentCode
#   ELSE vc.AgentCode
# END AS AgentCode,
# CASE
#   WHEN vc.AgentCode IN ('70007328', '70009250') THEN CONCAT(pa.Surname, ', ', pa.FirstName)
#   ELSE CONCAT(ag.Surname, ' ', ag.FirstName)
# END AS AgentName,
# vc.BranchName,
# CASE
#   WHEN o.POLICY_NUMBER IS NOT NULL THEN 'Yes'
#   ELSE 'No'
# END AS OrphanAccount,
# vc.PaidToDate,
# alt.PAID_TO_DATE,
# vc.DateDif,
# vc.BillFreq,
# vc.Currency,
# vc.InstPrem,
# vc.APE,
# vc.PayMode,
# cl.Email,
# cl.EmailValidityCheck,
# cl.HasValidContact,
# cl.MobileNo,
# cl.MobileNoCheck,
# cl.Telno1,
# cl.Telno2,
# vc.RiskStatus,
# vc.PremStatus,
# CASE
#   WHEN cl.Salutation IS NULL OR cl.Salutation = '' OR vc.PayMode = 'G' THEN 'Corporate'
#   ELSE 'Non-corporate'
# END AS ClientType,
# alt.REMARKS,
# alt.Decline_Category,
# alt.TRANSACTION_DATE AS LastTransactionDate,
# alt.STATUS,
# alt.BILLING_TYPE,
# alt.SOURCE
# FROM autopay_latesttransaction alt
# LEFT JOIN vwcontracts vc
# ON alt.POLICY_NUMBER = vc.PolicyNo
# LEFT JOIN clients cl
# ON vc.Owner = cl.ClientID
# LEFT JOIN (
#   SELECT
#   ag.AgentCode,
#   ag.ClientID,
#   cl.SurName,
#   cl.FirstName
#   FROM ext_scv_catalog.dbo.agents ag
#   LEFT JOIN clients cl
#   ON ag.ClientID = cl.ClientID
# ) ag
# ON vc.AgentCode = ag.AgentCode
# LEFT JOIN contracttype ct
# ON vc.ContractType = ct.ContractType
# LEFT JOIN PREVAGENT pa
# ON vc.PolicyNo = pa.PolicyNo
# LEFT JOIN orphan o
# ON alt.POLICY_NUMBER = o.POLICY_NUMBER
# )
# SELECT
# POLICY_NUMBER AS `Policy Number`,
# MultiplePolicies AS `Multiple Policies?`,
# Salutation AS `Salutation`,
# SurName AS `Policy Owner's Surname`,
# FirstName AS `Policy Owner's First name`,
# FirstRDC AS `Policy Issuance Date`,
# ConTypeLongDesc AS `Plan Name`,
# AgentCode AS `Agent Code`,
# AgentName AS `Current Agent`,
# BranchName AS `Current Branch`,
# -- PaidToDate AS `Due Date`,
# PAID_TO_DATE AS `Due Date`,
# Currency AS `Currency`,
# InstPrem AS `Premium Amount`,
# BillFreq AS `Payment Frequency`,
# Paymode AS `Payment Method`,
# OrphanAccount AS `Orphan Account`,
# Email AS `Email Address`,
# MobileNo AS `Mobile Number`,
# Telno1 AS `Telephone Number 1`,
# Telno2 AS `Telephone Number 2`,
# REMARKS AS `Reason of Unsuccessful Billing`,
# LastTransactionDate AS `Last Transaction Date`,
# Owner AS `OwnerID`
# FROM autodeclined
# WHERE HasValidContact = 1
# AND ClientType = 'Non-corporate'
# AND Decline_Category = 'Hard Decline'
# AND ProductGroup = 'UL'
# AND LastTransactionDate = '2025-07-20'
# AND BILLING_TYPE = 'Current Due'
# ORDER BY LastTransactionDate DESC

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW callout_campaign AS
WITH autopay_latesttransaction AS (
WITH cleaned_data AS (
  SELECT 
    TRIM(REPLACE(REPLACE(REPLACE(POLICY_NUMBER, ',' , ''), ' ', ''), '.', '')) AS POLICY_NUMBER,
    REMARKS,
    TRANSACTION_DATE,
    PAID_TO_DATE,
    STATUS,
    SOURCE,
    BILLING_TYPE
  FROM curated_catalog.analytics.autopaybilling_daily
),
label_latest_transaction AS (
  SELECT 
    POLICY_NUMBER AS POLICY_NUMBER,
    REMARKS,
    TRANSACTION_DATE,
    PAID_TO_DATE,
    STATUS,
    SOURCE,
    BILLING_TYPE,
    ROW_NUMBER() OVER (PARTITION BY POLICY_NUMBER ORDER BY TRANSACTION_DATE DESC) AS rn
  FROM cleaned_data
)
SELECT 
POLICY_NUMBER,
REMARKS,
TRANSACTION_DATE,
PAID_TO_DATE,
STATUS,
SOURCE,
BILLING_TYPE,
CASE
  WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'cybs' AND (
    REMARKS IS NULL OR
    LOWER(REMARKS) LIKE '%inactive card%' OR
    LOWER(REMARKS) LIKE '%invalid account number%' OR
    LOWER(REMARKS) LIKE '%stolen or lost card%' OR
    LOWER(REMARKS) LIKE '%no such issuer%' OR
    LOWER(REMARKS) LIKE '%invalid card brand' OR
    LOWER(REMARKS) LIKE '%invalid card brand china union pay%' OR
    LOWER(REMARKS) LIKE '%invalid card brand discover%' OR
    LOWER(REMARKS) LIKE '%invalid card brand household%' OR
    LOWER(REMARKS) LIKE '%blank token%' OR
    LOWER(REMARKS) LIKE '%the issuing bank has questions%'
  ) THEN 'Hard Decline'

  WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'cybs' AND (
    LOWER(REMARKS) LIKE '%api error%' OR
    LOWER(REMARKS) LIKE '%err001%' OR
    LOWER(REMARKS) LIKE '%err002%' OR
    LOWER(REMARKS) LIKE '%expired card%' OR
    LOWER(REMARKS) LIKE '%general decline%' OR
    LOWER(REMARKS) LIKE '%insufficient funds%' OR
    LOWER(REMARKS) LIKE '%invalid card verification number%' OR
    LOWER(REMARKS) LIKE '%credit limit%' OR
    LOWER(REMARKS) LIKE '%missing one or more fields%' OR
    LOWER(REMARKS) LIKE '%invalid amt%' OR
    LOWER(REMARKS) LIKE '%issuer inoperative%' OR
    LOWER(REMARKS) LIKE '%no savings account%' OR
    LOWER(REMARKS) LIKE '%system malfunction%' OR
    LOWER(REMARKS) LIKE '%violation of law%' OR
    LOWER(REMARKS) LIKE '%missing card brand%' OR
    LOWER(REMARKS) LIKE '%reference number is missing%'
  ) THEN 'Soft Decline'

  WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) = 'fiserv' THEN 'Soft Decline'
  WHEN LOWER(STATUS) = 'declined' AND LOWER(SOURCE) IN ('ada', 'aca') THEN 'Others'
  WHEN LOWER(STATUS) = 'declined' THEN 'Others'
  ELSE 'Others'
  END AS Decline_Category
FROM label_latest_transaction
WHERE rn = 1
),
-- Clients Table
clients AS (
SELECT
  clientID,
  Salutation,
  SurName,
  FirstName,
  Email,
  CASE 
    WHEN Email = '' THEN 0
    WHEN Email IS NULL THEN 0
    WHEN Email LIKE '%["(),:;<>\]%' THEN 0
    WHEN SUBSTRING(Email, CHARINDEX('@', Email), LEN(Email)) LIKE '%[!#$%&*+/=?^`_{|]%' THEN 0
    WHEN LEFT(Email, 1) LIKE '[-_.+]' OR RIGHT(Email, 1) LIKE '[-_.+]' THEN 0
    WHEN Email LIKE '%[%' OR Email LIKE '%]%' THEN 0
    WHEN Email LIKE '%@%@%' THEN 0
    WHEN Email NOT LIKE '_%@_%._%' THEN 0
    WHEN CHARINDEX('.', REVERSE(LTRIM(RTRIM(Email)))) < 2 THEN 0
    WHEN CHARINDEX('.@', Email) <> 0 AND CHARINDEX('..', Email) <> 0 THEN 0
    WHEN Email LIKE '%dummy@dummy.com%' THEN 0
    WHEN Email LIKE '%PLUK%' OR Email LIKE '%pluk%' OR Email LIKE '%Pluk%' THEN 0
    ELSE 1
  END AS EmailValidityCheck,
CASE 
  WHEN MobileNo IS NULL OR TRIM(MobileNo) IN ('', '0', 'NA', '-') THEN NULL
  ELSE MobileNo
END AS MobileNo,  
CASE 
  WHEN RIGHT(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10) RLIKE '^[0-9]{10}$' AND RIGHT(LPAD(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10, '0'), 10) <> '0000000000' THEN RIGHT(REGEXP_REPLACE(MobileNo, '[^0-9]', ''), 10)
  ELSE '9898'
END AS MobileNoCheck,
CASE 
  WHEN Telno1 IS NULL OR TRIM(Telno1) IN ('', '0', 'NA', '-') THEN NULL
  ELSE Telno1
END AS Telno1,
CASE 
  WHEN Telno2 IS NULL OR TRIM(Telno2) IN ('', '0', 'NA', '-') THEN NULL
  ELSE Telno2
END AS Telno2,
  CASE 
    WHEN (
      (Email IS NOT NULL AND Email != '' AND
        CASE 
          WHEN Email LIKE '%["(),:;<>\]%' THEN 0
          WHEN SUBSTRING(Email, CHARINDEX('@', Email), LEN(Email)) LIKE '%[!#$%&*+/=?^`_{|]%' THEN 0
          WHEN LEFT(Email, 1) LIKE '[-_.+]' OR RIGHT(Email, 1) LIKE '[-_.+]' THEN 0
          WHEN Email LIKE '%[%' OR Email LIKE '%]%' THEN 0
          WHEN Email LIKE '%@%@%' THEN 0
          WHEN Email NOT LIKE '_%@_%._%' THEN 0
          WHEN CHARINDEX('.', REVERSE(LTRIM(RTRIM(Email)))) < 2 THEN 0
          WHEN CHARINDEX('.@', Email) <> 0 AND CHARINDEX('..', Email) <> 0 THEN 0
          WHEN Email LIKE '%dummy@dummy.com%' THEN 0
          WHEN Email LIKE '%PLUK%' OR Email LIKE '%pluk%' OR Email LIKE '%Pluk%' THEN 0
          ELSE 1
        END = 1
      )
    OR (MobileNo IS NOT NULL AND TRIM(MobileNo) NOT IN ('', '0', 'NA', '-'))
    OR (Telno1 IS NOT NULL AND TRIM(Telno1) NOT IN ('', '0', 'NA', '-'))
    OR (Telno2 IS NOT NULL AND TRIM(Telno2) NOT IN ('', '0', 'NA', '-'))
    ) THEN 1
    ELSE 0
  END AS HasValidContact
FROM ext_scv_catalog.dbo.clients
),
-- Previous Agents
PREVAGENT AS (
    WITH CTE AS (
      SELECT 
        PolicyNo,
        AgentCode,
        CurrentFrom,
        ROW_NUMBER() OVER (PARTITION BY PolicyNo ORDER BY CurrentFrom DESC, COALESCE(CurrentTo, TIMESTAMP('2070-12-31 00:00:00')) DESC) AS rn
      FROM ext_scv_catalog.dbo.policyagenthistory
      WHERE AgentCode NOT IN ('70007328', '70009250')
    )
    SELECT 
      a.PolicyNo,
      a.AgentCode,
      b.ClientID,
      b.Surname,
      b.FirstName
    FROM CTE a
    LEFT JOIN (
      SELECT 
        ag.AgentCode, 
        ag.ClientID,
        cl.SurName, 
        cl.FirstName
      FROM ext_scv_catalog.dbo.agents ag
      LEFT JOIN clients cl 
      ON ag.ClientID = cl.ClientID
    ) b
    ON a.AgentCode = b.AgentCode
    WHERE a.rn = 1
),
-- vwcontracts Table
vwcontracts AS (
SELECT
DISTINCT
PolicyNo,
owner,
CAST(FirstRCD AS DATE) AS FirstRDC,
CASE
  WHEN ContractType IN ('YP5', 'YP7', 'YPT', 'YPF', 'YD5', 'YD7', 'YDT', 'YDF', 'XP5', 'XP7', 'XPT', 'XPF', 'XLT', 'XLF', 'XD5', 'XD7', 'XDT', 'XDF', 'R05', 'R07', 'R10', 'R15', 'N05', 'N07', 'N10', 'N15', 'D05', 'D07', 'D10', 'D15', 'C20', 'RLP', 'RDP', 'RPA', 'RPB', 'RPD', 'PHP', 'PHD', 'PIA', 'PI2', 'P3P', 'P3D', 'P3L', 'P3M', 'RPW', 'RW1', 'RW2', 'CAP', 'YP2') 
  THEN 'UL'
  WHEN ContractType IN ('E20', 'AAA', 'PRO', 'PLP', 'BCB', 'BCE', 'EWR', 'CB1', 'CB2', 'CB3', 'CB4', 'CB5', 'ADP', 'DHI', 'TLR', 'ACP', 'LCP', 'MCP', 'PPA', 'LPB', 'LPC', 'LPD', 'LPA', 'W5P', 'W6P', 'PPP', 'WPP', 'LH1', 'LH2', 'LH3', 'LH4', 'PCA', 'D12', 'D06', 'P12', 'P06', 'PDL', 'E2S', 'PFL', 'PLD', 'PLL', 'P5L', 'PHA', 'HG1', 'HG2', 'HG3', 'PLB', 'PPL', 'W2P', 'WPL', 'W4P', 'I12', 'I06', 'P1S', 'P2S', 'W2A', 'P3S', 'PS0', 'PS2', 'PS3', 'PS4', 'PS6', 'PS7', 'PS8', 'PS1', 'PS5', 'PS9', 'P5P', 'PP5', 'W1P', 'WP5', 'PW1', 'PW2', 'PW3', 'PW4', 'RRI', 'CTM', 'STM', 'HTM', 'CT3', 'CTR', 'T15', 'T90', 'WLR', 'WLU', 'SPA', 'LI5', 'LIT')
  THEN 'Trad'
END AS ProductGroup,
CAST(PaidToDate AS DATE) AS PaidToDate,
DATEDIFF(MONTH, PaidToDate, CURRENT_DATE()) AS Datedif,
CAST(BillToDate AS DATE) AS BillToDate,
CASE
  WHEN BillFreq = 1 THEN 'Annual'
  WHEN BillFreq = 2 THEN 'Semi-annual'
  WHEN BillFreq = 4 THEN 'Quarterly'
  WHEN BillFreq = 12 THEN 'Monthly'
  ELSE 'Single Pay'
END AS BillFreq,
Currency,
InstPrem,
InstPrem * BillFreq AS APE,
CASE
  WHEN PayMode = 'C' THEN 'Cash'
  WHEN PayMode = 'D' THEN 'Auto-debit'
  WHEN PayMode = 'K' THEN 'Auto-credit'
  WHEN PayMode = 'T' THEN 'Auto-pay'
  WHEN PayMode = 'G' THEN 'Group'
  ELSE 'Others'
END AS PayMode,
InstPrem,
RiskStatus,
PremStatus,
COUNT(PolicyNo) OVER (PARTITION BY Owner) AS PolicyCount,
AgentCode,
ContractType,
BranchName
FROM ext_scv_catalog.dbo.vwcontracts
WHERE Owner NOT IN (
  SELECT
  Owner
  FROM ext_scv_catalog.dbo.vwcontracts
  WHERE PolicyNo IN (
  SELECT
  Policy_Number
  FROM curated_catalog.analytics.call_center_dnc
  )
)
),
contracttype AS (
SELECT
TRIM(ContractType) AS ContractType,
TRIM(ConTypeShortDesc) AS ConTypeShortDesc,
TRIM(ConTypeLongDesc) AS ConTypeLongDesc
FROM ext_scv_catalog.dbo.contracttype
),
-- Orphan Accounts
orphan AS (
SELECT 
TRIM(REPLACE(REPLACE(REPLACE(CHDRNUM, ',' , ''), ' ', ''), '.', '')) AS POLICY_NUMBER
FROM ds_catalog.str_lifeasia.zchdpf
WHERE isValid = 1 AND ORPTAG = 'ORP' 
),
-- Latest Transaction -> Total Count: 745544
-- Main Table - Auto Decline Data Extraction
autodeclined AS (
SELECT
alt.POLICY_NUMBER,
CASE
  WHEN vc.PolicyCount > 1 THEN 'Yes'
  ELSE 'No'
END AS MultiplePolicies,
vc.Owner,
cl.Salutation,
cl.SurName,
cl.FirstName,
vc.FirstRDC,
ct.ConTypeLongDesc,
vc.ProductGroup,
CASE
  WHEN vc.AgentCode IN ('70007328', '70009250') THEN pa.AgentCode
  ELSE vc.AgentCode
END AS AgentCode,
CASE
  WHEN vc.AgentCode IN ('70007328', '70009250') THEN CONCAT(pa.Surname, ', ', pa.FirstName)
  ELSE CONCAT(ag.Surname, ' ', ag.FirstName)
END AS AgentName,
vc.BranchName,
CASE
  WHEN o.POLICY_NUMBER IS NOT NULL THEN 'Yes'
  ELSE 'No'
END AS OrphanAccount,
vc.PaidToDate,
alt.PAID_TO_DATE,
vc.DateDif,
vc.BillFreq,
vc.Currency,
vc.InstPrem,
vc.APE,
vc.PayMode,
cl.Email,
cl.EmailValidityCheck,
cl.HasValidContact,
cl.MobileNo,
cl.MobileNoCheck,
cl.Telno1,
cl.Telno2,
vc.RiskStatus,
vc.PremStatus,
CASE
  WHEN cl.Salutation IS NULL OR cl.Salutation = '' OR vc.PayMode = 'G' THEN 'Corporate'
  ELSE 'Non-corporate'
END AS ClientType,
alt.REMARKS,
alt.Decline_Category,
alt.TRANSACTION_DATE AS LastTransactionDate,
alt.STATUS,
alt.BILLING_TYPE,
alt.SOURCE
FROM autopay_latesttransaction alt
LEFT JOIN vwcontracts vc
ON alt.POLICY_NUMBER = vc.PolicyNo
LEFT JOIN clients cl
ON vc.Owner = cl.ClientID
LEFT JOIN (
  SELECT
  ag.AgentCode,
  ag.ClientID,
  cl.SurName,
  cl.FirstName
  FROM ext_scv_catalog.dbo.agents ag
  LEFT JOIN clients cl
  ON ag.ClientID = cl.ClientID
) ag
ON vc.AgentCode = ag.AgentCode
LEFT JOIN contracttype ct
ON vc.ContractType = ct.ContractType
LEFT JOIN PREVAGENT pa
ON vc.PolicyNo = pa.PolicyNo
LEFT JOIN orphan o
ON alt.POLICY_NUMBER = o.POLICY_NUMBER
)
SELECT
POLICY_NUMBER AS `Policy Number`,
MultiplePolicies AS `Multiple Policies?`,
Salutation AS `Salutation`,
SurName AS `Policy Owner's Surname`,
FirstName AS `Policy Owner's First name`,
FirstRDC AS `Policy Issuance Date`,
ConTypeLongDesc AS `Plan Name`,
AgentCode AS `Agent Code`,
AgentName AS `Current Agent`,
BranchName AS `Current Branch`,
-- PaidToDate AS `Due Date`,
PAID_TO_DATE AS `Due Date`,
Currency AS `Currency`,
InstPrem AS `Premium Amount`,
BillFreq AS `Payment Frequency`,
Paymode AS `Payment Method`,
OrphanAccount AS `Orphan Account`,
Email AS `Email Address`,
MobileNo AS `Mobile Number`,
Telno1 AS `Telephone Number 1`,
Telno2 AS `Telephone Number 2`,
REMARKS AS `Reason of Unsuccessful Billing`,
LastTransactionDate AS `Last Transaction Date`,
Owner AS `OwnerID`
FROM autodeclined
WHERE HasValidContact = 1
AND ClientType = 'Non-corporate'
AND Decline_Category = 'Hard Decline'
AND ProductGroup = 'UL'
AND LastTransactionDate = DATE_ADD(DATE(FROM_UTC_TIMESTAMP(CURRENT_TIMESTAMP(), 'Asia/Manila')), -1)
AND BILLING_TYPE = 'Current Due'
ORDER BY LastTransactionDate DESC

In [0]:
%sql
SELECT *
FROM callout_campaign

Policy Number,Multiple Policies?,Salutation,Policy Owner's Surname,Policy Owner's First name,Policy Issuance Date,Plan Name,Agent Code,Current Agent,Current Branch,Due Date,Currency,Premium Amount,Payment Frequency,Payment Method,Orphan Account,Email Address,Mobile Number,Telephone Number 1,Telephone Number 2,Reason of Unsuccessful Billing,Last Transaction Date,OwnerID
12998125,Yes,MR,SY,MICHAEL LIM,2020-10-20,PRULINK ELITE PROTECTOR 7,70000049,Huang Peir Pyng Marilou,ALEXANDRITE PREMIER LIA,2025-07-20,PHP,16667.00,Monthly,Auto-pay,No,VCKYSY@YAHOO.COM,092099037382,0283521873,0283521873,STOLEN OR LOST CARD.,2025-07-20,82352772
02686319,No,MRS,CABILING,MARIA RITA ROSSARI SUANES,2009-03-20,PRULINK ASSURANCE ACCOUNT PLUS,70000049,Huang Peir Pyng Marilou,ALEXANDRITE PREMIER LIA,2025-07-20,PHP,3000.00,Monthly,Auto-pay,No,rscabiling@yahoo.com,09178354183,9316990,4629180,INACTIVE CARD OR CARD NOT AUTHORIZED FOR CARD NOT PRESENT TRANSACT…,2025-07-20,80793695
12489551,No,MRS,YALONG,ROMILYN PANAO,2020-06-20,PRULINK ASSURANCE ACCOUNT PLUS,70060011,JUAN MARTIN JOHN,DIAMOND 2.0 BRANCH,2025-07-20,PHP,3022.71,Monthly,Auto-pay,No,romilyn.yalong@gmail.com,09065945070,09065945070,09065945070,INVALID ACCOUNT NUMBER,2025-07-20,82821825
17707478,Yes,MRS,ABELLAR,MARILYN DURAN,2022-07-20,PRULINK EXACT PROTECTOR 10,70043922,ABELLAR MARILYN DURAN,ALEXANDRITE PREMIER LIA,2025-07-20,PHP,1500.00,Monthly,Auto-pay,No,marilynabellar@yahoo.com,09178582245,09178582245,09178582245,STOLEN OR LOST CARD.,2025-07-20,81428973
12386978,No,MRS,VELO,SARA JOY ROSARIO,2020-06-20,PRULINK ASSURANCE ACCOUNT PLUS,70044362,PALGUE ADRIAN BADUYEN,TITANIUM 2 LIFE INS. AGY INC,2025-07-20,PHP,1500.00,Monthly,Auto-pay,No,SARA.SINGIAN@GMAIL.COM,09399901279,null,null,INACTIVE CARD OR CARD NOT AUTHORIZED FOR CARD NOT PRESENT TRANSACT…,2025-07-20,82967819
24745081,Yes,MRS,YU,SHARA SANTOS,2024-10-20,PRULINK ASSURANCE ACCOUNT PLUS,70142316,BUGARIN ANGELINE LAUD,TITANIUM BRANCH,2025-07-20,PHP,7800.00,Quarterly,Auto-pay,No,SHARAYU0514@GMAIL.COM,09063466514,09063466514,09063466514,null,2025-07-20,82021055
23530989,No,MS,ABECIA,MARIE ANTONETTE PARAS,2024-04-20,PRULINK ASSURANCE ACCOUNT PLUS,70109071,PADAYHAG LENDYL BUCTON,CHINA GOLD SOUTHERN SUMMIT,2025-07-20,PHP,2500.00,Monthly,Auto-pay,No,cbmarieantonette@gmail.com,09465542670,09465542670,09465542670,INACTIVE CARD OR CARD NOT AUTHORIZED FOR CARD NOT PRESENT TRANSACT…,2025-07-20,85135287
25752228,No,MR,CORTEZ,ALVIN GALANG,2025-04-20,PRULINK ASSURANCE ACCOUNT PLUS,70113459,LIONG JEANNEL CABACCANG,CRYSTAL QUARTZ BRANCH,2025-07-20,PHP,2615.00,Monthly,Auto-pay,No,binbincortez02@gmail.com,09560662385,9560662385,9560662385,null,2025-07-20,85539151
19048684,Yes,MS,CHUA,KIM NICOLE SANTIAGO,2022-10-20,PRULINK ELITE PROTECTOR 10,70124953,CHUA KIM NICOLE SANTIAGO,RED LION SUMMIT LIFE INS AGY,2025-07-20,PHP,9200.00,Monthly,Auto-pay,No,plukkimnicolechua@gmail.com,9175230866,9175230866,9175230866,STOLEN OR LOST CARD.,2025-07-20,82777156
25891925,No,MS,MAAGAD,RICHELLE PAGE,2025-05-20,PRULINK ASSURANCE ACCOUNT PLUS,70176647,CAMEYOG MATTHEW CULLEN ESCOLANO,CHINA GOLD SOUTHERN SUMMIT,2025-07-20,PHP,2533.09,Monthly,Auto-pay,No,megadark.pagemaagad@gmail.com,09554864579,09554864579,09554864579,INACTIVE CARD OR CARD NOT AUTHORIZED FOR CARD NOT PRESENT TRANSACT…,2025-07-20,85569392


In [0]:
%pip install openpyxl
%pip install fsspec
%pip install pyexcelerate

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
from pyspark.sql.functions import coalesce, col, lit,when,current_timestamp, from_utc_timestamp

In [0]:
databricks_instance_dict = {
  'adb-774780659964517.17.azuredatabricks.net' : 'dev',
  'adb-8734470252293229.9.azuredatabricks.net' : 'uat',
  'adb-5627953250658255.15.azuredatabricks.net' : 'prd',
}
storage_account_dict = {
  'dev': 'staphlifedevaz1tl3i9a002',
  'uat': 'staphlifeuataz1n1ukje001',
  'prd': 'staphlifeprdaz1g8upqt001'
}
catalog_prefix_dict = {
  'dev': 'dev_',
  'uat': 'uat_',
  'prd': ''
}

env = databricks_instance_dict[spark.conf.get('spark.databricks.workspaceUrl')]
storageAccountName = storage_account_dict[env]
catalogPrefix = catalog_prefix_dict[env]

In [0]:
secretScope = "adls-akv-master"
spIdSecret = "pluk-dl-spn-clientid"
spKeySecret = "pluk-dl-spn-clientsecret"
sourceFileSystem = "raw"
sinkFileSystem = "structured"

In [0]:
spId = dbutils.secrets.get(scope="adls-akv-master", key="pluk-dl-spn-clientid")
spKey = dbutils.secrets.get(scope="adls-akv-master", key="pluk-dl-spn-clientsecret")

tenantId = dbutils.secrets.get(scope="adls-akv-master", key="pluk-dl-tenant-id")

In [0]:
spark.conf.set("fs.azure.account.auth.type", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id", spId)
spark.conf.set("fs.azure.account.oauth2.client.secret", spKey)
spark.conf.set("fs.azure.account.oauth2.client.endpoint", "https://login.microsoftonline.com/" + tenantId + "/oauth2/token")

In [0]:
%python
# Create the directory if it does not exist
import os

local_dir = "/tmp/data_extracts"
if not os.path.exists(local_dir):
    os.makedirs(local_dir)

In [0]:
%python
# Ensure the necessary libraries are installed
# Protect the Excel file
from openpyxl import load_workbook
from openpyxl.styles import Protection
import pandas as pd

# Load the data from the view into a Spark DataFrame
df = spark.sql("SELECT * FROM callout_campaign")

# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = df.toPandas()

# Get the current date - 1 (T-1) in the required format
current_date_str = spark.sql("SELECT date_format(date_sub(current_date(), 1), 'MM-dd-yy')").collect()[0][0]
for_email_date = spark.sql("SELECT date_format(date_sub(current_date(), 1), 'MM-dd-yy')").collect()[0][0]

### CHANGE THE NAME OF THE PATH WHEN PERFORMING THIS CODE
# Save the Pandas DataFrame as an Excel file to a local path
local_path = f"/tmp/data_extracts/Auto_Decline_Callout_Campaign_{current_date_str}.xlsx"
pandas_df.to_excel(local_path, index=False)

wb = load_workbook(local_path)
ws = wb.active

# Lock all cells
for row in ws.iter_rows():
    for cell in row:
        cell.protection = Protection(locked=True)

ws.protection.sheet = True
ws.protection.password = 'callout_campaign'
ws.protection.enable()
ws.protection.selectLockedCells = False
ws.protection.selectUnlockedCells = False
ws.protection.formatCells = False
ws.protection.formatColumns = False
ws.protection.formatRows = False
ws.protection.insertColumns = False
ws.protection.insertRows = False
ws.protection.insertHyperlinks = False
ws.protection.deleteColumns = False
ws.protection.deleteRows = False
ws.protection.sort = False
ws.protection.autoFilter = False
ws.protection.pivotTables = False
ws.protection.objects = True
ws.protection.scenarios = True

# Save the workbook with a password to open
wb.security.workbookPassword = 'callout_campaign'
wb.security.lockStructure = True
wb.security.lockWindows = True
wb.security.revisionsPassword = 'callout_campaign'
wb.save(local_path)

### OPTIONAL: SAVING EXCEL TO DBFS
# # Move the file to the permanent storage location
# dbutils.fs.cp(f"file:{local_path}", f"/FileStore/data_extracts/gross_premium_extract_{current_date_str}_test_only.xlsx")

# Send the Excel file via Outlook email
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText

subject = f"Auto Decline Callout Campaign - {for_email_date}"
body = f"""
<html>
  <body>
    <p><strong>Note: This is an automated email. Please do not reply.</strong></p>
    <p>I hope this email finds you well.</p>
    <p>Please find attached the Excel file extract as requested. This file contains the data extracted from the Unified Data Platform (UDP).</p>
    <p>If you encounter any issues or have any concerns regarding the data extracted, please do not hesitate to let us know. We are more than happy to review the extract and clarify any adjustments needed in the transformation of the data.</p>
    <p>Thank you for your attention to this matter.</p>
    <p>Best regards,<br>Data &amp; AI</p>
  </body>
</html>
"""

# Define sender, recipients, and CC
sender_email = "admphlifeprddldtb@prulifeuk.com.ph"
receiver_emails = ["Mitchie.R.Salvana@prulifeuk.com.ph", "charisse.r.briones@prulifeuk.com.ph"]
cc_emails = ["jerome.pintucan.vertere@prulifeuk.com.ph"]

# Create the email message
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = ", ".join(receiver_emails)
msg['Cc'] = ", ".join(cc_emails)
msg['Subject'] = subject
msg.attach(MIMEText(body, 'html')) # plain, html

# Attach the Excel file
with open(local_path, "rb") as attachment:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f"attachment; filename= {local_path.split('/')[-1]}")
    msg.attach(part)

# Send the email
try:
    with smtplib.SMTP('mailsg.intranet.asia', 25) as server:
        server.starttls()
        all_recipients = receiver_emails + cc_emails
        server.sendmail(sender_email, all_recipients, msg.as_string())
        print("Email sent successfully!")
except Exception as e:
    print(f"Error sending email: {e}")


Email sent successfully!


In [0]:
%python
dbutils.notebook.exit("Exiting notebook")